# Example of bus arrival time prediction with DNN
### Author: Wichai T.
This notebook is an example of using DNNRegressor from TensorFlow package.  
The DNN is used to learn data of bus travel time to a stop, then the learned model is used to predict the bus travel time with other given set of parameters.

In [1]:
# importing packages

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from math import sqrt

## Read data from a csv file

In [2]:
df = pd.read_csv(r'data\bus_traveling_data.csv', header=0)

print('Shape of data: ', df.shape)

Shape of data:  (241, 15)


In [3]:
# show some sample of data
df.head()

,t_bc,t_bc_ff,t_bc_extp,t_ab,link_progress,speed,sms_ab,exponential_decayed_speed,two_gps_space_mean_speed,six_gps_space_mean_speed,eighteen_gps_space_mean_speed,thirty_gps_space_mean_speed,sixty_gps_space_mean_speed,ninety_gps_space_mean_speed,BC_distance_in_meter_by_linear_ref
0,20,29,0.0,0,0.089130,33,33.000000,31.886683,33.1104,27.4518,25.6430,21.7842,12.6122,8.4081,224.063441
1,10,17,11.0,10,0.494944,32,43.830200,31.915012,34.1690,27.8687,27.4193,22.4323,13.1760,8.7840,124.237845
2,30,60,768.0,10,0.012866,16,1.139332,21.727783,8.6832,21.2578,25.2992,18.9315,11.1607,7.4404,242.823587
3,20,42,47.0,20,0.302005,30,13.372144,23.795837,24.6647,19.2013,25.8253,19.6032,11.5677,7.7118,171.698711
4,10,18,13.0,30,0.713728,35,21.068264,26.596878,34.7717,18.9430,26.4936,20.5449,12.1415,8.0943,70.419530


## Split train / test data (70:30%)

In [4]:
train_df = df.iloc[:round(len(df)*.7)]
test_df = df.iloc[round(len(df)*.7)+1:]

In [5]:
# select a label and features
FEATURES = df.columns[~df.columns.isin(['t_bc','stop_pair','A_stop_id','C_stop_id'])].tolist()
LABEL = "t_bc"

In [6]:
def data_init(data_set):
    feature_cols = {k: tf.constant(data_set[k].values) for k in FEATURES}
    labels = tf.constant(data_set[LABEL].values)
    return feature_cols, labels

## Create DNN model and train

In [7]:
# Initializes a DNNRegressor instance with 4 hidden layer, each 30 nodes
estimator = tf.contrib.learn.DNNRegressor(
    feature_columns=[tf.contrib.layers.real_valued_column(k) for k in FEATURES],
    hidden_units=[30, 30, 30, 30])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'tf_random_seed': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000004C707706A0>, '_task_id': 0, '_task_type': None, 'save_summary_steps': 100, 'save_checkpoints_secs': 600, '_master': '', '_num_ps_replicas': 0, '_environment': 'local', '_evaluation_master': '', 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, 'keep_checkpoint_every_n_hours': 10000, 'keep_checkpoint_max': 5, 'save_checkpoints_steps': None}


In [8]:
# training
estimator.fit(input_fn=lambda: data_init(train_df), max_steps=5000)

INFO:tensorflow:Summary name dnn/hiddenlayer_0:fraction_of_zero_values is illegal; using dnn/hiddenlayer_0_fraction_of_zero_values instead.
INFO:tensorflow:Summary name dnn/hiddenlayer_0:activation is illegal; using dnn/hiddenlayer_0_activation instead.
INFO:tensorflow:Summary name dnn/hiddenlayer_1:fraction_of_zero_values is illegal; using dnn/hiddenlayer_1_fraction_of_zero_values instead.
INFO:tensorflow:Summary name dnn/hiddenlayer_1:activation is illegal; using dnn/hiddenlayer_1_activation instead.
INFO:tensorflow:Summary name dnn/hiddenlayer_2:fraction_of_zero_values is illegal; using dnn/hiddenlayer_2_fraction_of_zero_values instead.
INFO:tensorflow:Summary name dnn/hiddenlayer_2:activation is illegal; using dnn/hiddenlayer_2_activation instead.
INFO:tensorflow:Summary name dnn/hiddenlayer_3:fraction_of_zero_values is illegal; using dnn/hiddenlayer_3_fraction_of_zero_values instead.
INFO:tensorflow:Summary name dnn/hiddenlayer_3:activation is illegal; using dnn/hiddenlayer_3_acti

DNNRegressor(dropout=None, optimizer=None, feature_columns=[_RealValuedColumn(column_name='t_bc_ff', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='t_bc_extp', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='t_ab', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='link_progress', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='speed', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='sms_ab', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='exponential_decayed_speed', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='two_gps_space_mean_speed', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(colum

## Predict

In [9]:
y_predict = estimator.predict(input_fn=lambda: data_init(test_df), as_iterable=False)

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in t

## Evaluate

In [10]:
y_test = test_df[LABEL].values
mae = np.mean([abs(true-predict) for true,predict in zip(y_test, y_predict)])
rmse = sqrt(mean_squared_error(y_test, y_predict))
mape = np.mean([abs(true-predict)/true for true,predict in zip(y_test, y_predict)])*100

print('Mean absolute error (MAE): \t\t%.2f sec' % mae)
print('Root mean square error (RMSE): \t\t%.2f sec' % rmse)
print('Mean absolute percentage error (MAPE): \t%.2f %%' % mape)

Mean absolute error (MAE): 		2.33 sec
Root mean square error (RMSE): 		3.50 sec
Mean absolute percentage error (MAPE): 	14.80 %


## Compare with OLS

In [11]:
print('MAE improve:  %.2f %%' % ((2.82-mae)/.0282))
print('RMSE improve: %.2f %%' % ((3.46-rmse)/.0346))
print('MAPE improve: %.2f %%' % (20.67-mape))

MAE improve:  17.23 %
RMSE improve: -1.22 %
MAPE improve: 5.87 %
